In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 부모 디렉토리 추가 및 커스텀 모듈 임포트
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from multilayer_model.m3_hierarchical import HierarchicalCVAE3
from multilayer_loss.l_multi3_hierarchical import l_multi3_hierarchical, l_multi3_bce_hierarchical
from vae_earlystopping import EarlyStopping

In [2]:
# 1. 데이터 로드 (사용자 지정 경로 및 파일명 반영)
try:
    x1_data = np.load('../data/metal.npy')             # Phase 1: Metal
    x2_data = np.load('../data/support_norm.npy')      # Phase 2: Support
    x3_data = np.load('../data/pre_fin.npy')      # Phase 3: Pretreatment
    c_data = np.load('../data/re_fin.npy')             # Condition: Reaction/Active
    print("✅ All 3-Phase data loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")

# 2. 데이터 분할 (Train/Val/Test) - 인덱스 일관성 유지
indices = np.arange(len(x1_data))
train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

def split_and_scale(data, train_idx, val_idx, test_idx, scale=True):
    train, val, test = data[train_idx], data[val_idx], data[test_idx]
    if scale:
        scaler = MinMaxScaler()
        train = scaler.fit_transform(train)
        val = scaler.transform(val)
        test = scaler.transform(test)
        return train, val, test, scaler
    return train, val, test, None

x1_train, x1_val, x1_test, _ = split_and_scale(x1_data, train_idx, val_idx, test_idx)
x2_train, x2_val, x2_test, _ = split_and_scale(x2_data, train_idx, val_idx, test_idx)
x3_train, x3_val, x3_test, _ = split_and_scale(x3_data, train_idx, val_idx, test_idx)
c_train, c_val, c_test, c_scaler = split_and_scale(c_data, train_idx, val_idx, test_idx)

# 3. 텐서 변환 및 데이터로더
def to_t(arr): return torch.tensor(arr, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(to_t(x1_train), to_t(x2_train), to_t(x3_train), to_t(c_train)), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(to_t(x1_val), to_t(x2_val), to_t(x3_val), to_t(c_val)), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(to_t(x1_test), to_t(x2_test), to_t(x3_test), to_t(c_test)), batch_size=64, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

✅ All 3-Phase data loaded successfully.
Using device: cuda


In [3]:
# 4. 3단계 계층 모델 초기화
x_dims = [x1_train.shape[1], x2_train.shape[1], x3_train.shape[1]]
c_dim = c_train.shape[1]
z_dims = [16, 8, 4] # Metal -> Support -> Pretreatment 순으로 latent 압축

model = HierarchicalCVAE3(x_dims, c_dim, z_dims).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
es = EarlyStopping(patience=50, min_delta=1e-7)

In [4]:
# 5. 학습 루프 (KL Annealing 적용)
epochs = 500
total_anneal_steps = 150 # 150에폭 동안 KL 가중치를 서서히 증가

for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0
    for b1, b2, b3, bc in train_loader:
        b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
        
        optimizer.zero_grad()
        out = model(b1, b2, b3, bc)
        
        # 3단계 계층적 Loss 계산 (BCE for Metal, MSE for others)
        # 여기서는 복합적인 학습을 위해 수치 복원(MSE) 위주로 구성된 l_multi3_hierarchical 사용
        # 실제 금속 존재 여부가 중요하면 l_multi3_bce_hierarchical로 교체 가능
        loss_results = l_multi3_hierarchical(
            out['x2_hat'], b2, # 메인 타겟 예시 (Support)
            out['mu_list'], out['lv_list'],
            gamma_list=[0.1, 0.05, 0.02], # NVAE 기반 가중치 차등
            step=epoch,
            total_steps=total_anneal_steps
        )
        
        loss = loss_results['loss']
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    v_loss = 0
    with torch.no_grad():
        for v1, v2, v3, vc in val_loader:
            v1, v2, v3, vc = v1.to(device), v2.to(device), v3.to(device), vc.to(device)
            vout = model(v1, v2, v3, vc)
            v_loss += l_multi3_hierarchical(
                vout['x2_hat'], v2, vout['mu_list'], vout['lv_list'],
                step=epoch, total_steps=total_anneal_steps
            )['loss'].item()
            
    avg_v_loss = v_loss / len(val_loader)
    if epoch % 20 == 0:
        print(f"Epoch {epoch}/{epochs} | V_Loss: {avg_v_loss:.6f} | Anneal_W: {loss_results['anneal_w']:.2f}")
    
    if es(avg_v_loss, model):
        print(f"Early stopping at epoch {epoch}")
        break

es.load_best_model(model)

EarlyStopping counter: 1 out of 50
Epoch 20/500 | V_Loss: 0.025761 | Anneal_W: 0.13
EarlyStopping counter: 1 out of 50
EarlyStopping counter: 2 out of 50
EarlyStopping counter: 1 out of 50
EarlyStopping counter: 2 out of 50
EarlyStopping counter: 3 out of 50
EarlyStopping counter: 4 out of 50
EarlyStopping counter: 1 out of 50
EarlyStopping counter: 2 out of 50
EarlyStopping counter: 1 out of 50
EarlyStopping counter: 2 out of 50
EarlyStopping counter: 3 out of 50
EarlyStopping counter: 4 out of 50
EarlyStopping counter: 5 out of 50
EarlyStopping counter: 6 out of 50
EarlyStopping counter: 7 out of 50
EarlyStopping counter: 8 out of 50
EarlyStopping counter: 9 out of 50
Epoch 40/500 | V_Loss: 0.022330 | Anneal_W: 0.27
EarlyStopping counter: 10 out of 50
EarlyStopping counter: 11 out of 50
EarlyStopping counter: 12 out of 50
EarlyStopping counter: 13 out of 50
EarlyStopping counter: 14 out of 50
EarlyStopping counter: 15 out of 50
EarlyStopping counter: 16 out of 50
EarlyStopping counte

In [ ]:
# 6. 생성 능력 검증 (Inference)
print("\n--- Generating New Catalyst Recipes ---")
model.eval()
with torch.no_grad():
    # 수정 부분: numpy 배열을 torch tensor로 변환 후 device로 이동
    sample_c = torch.tensor(c_test[0:1], dtype=torch.float32).to(device)
    
    # 모델의 sample 메서드 호출
    m_prob, s_hat, p_hat = model.sample(sample_c, num_samples=1, device=device)
    
    print("Generated Metal Presence (Prob):\n", m_prob.cpu().numpy())
    print("Generated Support Composition:\n", s_hat.cpu().numpy())
    print("Generated Pretreatment Parameters:\n", p_hat.cpu().numpy())

# 결과 저장
torch.save(model.state_dict(), '../model/h_cvae_3phase_best.pth')
print("\n✅ Best model saved to model/h_cvae_3phase_best.pth")


--- Generating New Catalyst Recipes ---


AttributeError: 'numpy.ndarray' object has no attribute 'to'